In [107]:
import pandas as pd
import numpy as np
import pickle

In [150]:
ball2ball = pd.read_csv('../data/ipl_ball_by_ball_2008_2022.csv')

In [83]:
ball2ball.columns

Index(['id', 'innings', 'overs', 'ball_number', 'batter', 'bowler',
       'non_striker', 'extra_type', 'batsman_run', 'extras_run', 'total_run',
       'non_boundary', 'iswicket_delivery', 'player_out', 'dismisal_kind',
       'fielders_involved', 'batting_team'],
      dtype='object')

In [84]:
matches = pd.read_csv('../data/ipl_matches_2008_2022.csv')

In [109]:
matches.columns

Index(['id', 'city', 'match_date', 'season', 'match_number', 'team1', 'team2',
       'venue', 'toss_winner', 'toss_decision', 'superover', 'winning_team',
       'won_by', 'margin', 'method', 'player_of_match', 'umpire1', 'umpire2'],
      dtype='object')

In [86]:
matches = matches.dropna(subset = 'match_date')

run this to add year column to ball2ball dataframe

In [151]:
import pandas as pd

# Assuming your original DataFrame is named 'original_df'
# Example:
# original_df = pd.read_csv('your_csv_file.csv')

# Create a new DataFrame with 'id' and 'year'
new_df = matches[['id', 'match_date']].copy()

# Convert 'match_date' to datetime format
new_df['match_date'] = pd.to_datetime(new_df['match_date'], format='%d-%m-%Y')

# Extract the year and create a new 'year' column
new_df['year'] = new_df['match_date'].dt.year

# Drop the 'match_date' column if you no longer need it
new_df.drop('match_date', axis=1, inplace=True)

# Print or use the new DataFrame 'new_df'
ball2ball = pd.merge(new_df, ball2ball, on='id', how='inner')


In [88]:
ball2ball['extra_type'].unique()

array([nan, 'legbyes', 'wides', 'byes', 'noballs', 'penalty'],
      dtype=object)

In [89]:
from collections import defaultdict
bowler_stats = {}
for index, row in ball2ball.iterrows():
    bowler = row['bowler']
    year = row['year']
    extra_type = row['extra_type']
    total_run = row['total_run']
    extras_run = row['extras_run']
    if year not in bowler_stats:
        bowler_stats[year] = {}
    if bowler not in bowler_stats[year]:
        bowler_stats[year][bowler] = {}
        bowler_stats[year][bowler]['runs_conceeded'] = 0
        bowler_stats[year][bowler]['balls_bowled'] = 0
    bowler_stats[year][bowler]['runs_conceeded'] += total_run - extras_run if extra_type in ['byes','legbyes'] else total_run
    bowler_stats[year][bowler]['balls_bowled'] += 1 if extra_type not in ['wides', 'noballs', 'penalty'] else 0
            


In [90]:
bowler_economy = bowler_stats
for year in bowler_economy:
    for player in bowler_economy[year]:
        bowler_economy[year][player]['economy'] = bowler_economy[year][player]['runs_conceeded'] * 6 / bowler_economy[year][player]['balls_bowled']

In [ ]:
best_economies = {}
for year in bowler_economy:
    sorted_economies = sorted(bowler_economy[year], key = lambda x: bowler_economy[year][x]['economy'])
    best5 = sorted_economies[0:5]
    best_economies[year] = {player: round(bowler_economy[year][player]['economy'],2) for player in best5}
    
best_economies

In [92]:
import pickle
with open('best_economies.pkl','wb') as bef:
    pickle.dump(best_economies,bef)

In [93]:
ball2ball['dismisal_kind'].unique()

array([nan, 'caught', 'caught and bowled', 'run out', 'bowled', 'stumped',
       'lbw', 'hit wicket', 'retired hurt', 'retired out',
       'obstructing the field'], dtype=object)

In [94]:
grouped_df = ball2ball.groupby(['year', 'batter'])['batsman_run'].sum().reset_index()
top_scorers_df = grouped_df.groupby('year').apply(lambda x: x.nlargest(5, 'batsman_run')).reset_index(drop=True)
# Print or use the resulting DataFrame 'grouped_df'
# top_scorers_dict = top_scorers_df.to_dict(orient='records')

# Print or use the resulting dictionary 'top_scorers_dict'
top_scorers_dict_2d = {}

# Iterate through the rows of the DataFrame and populate the 2D dictionary
for index, row in top_scorers_df.iterrows():
    year = row['year']
    batter = row['batter']
    batsman_run = row['batsman_run']

    # If the year is not in the dictionary, add it
    if year not in top_scorers_dict_2d:
        top_scorers_dict_2d[year] = {}

    # Add the batter and batsman_run to the dictionary for the specific year
    top_scorers_dict_2d[year][batter] = batsman_run

# Print or use the resulting 2D dictionary 'top_scorers_dict_2d'
print(top_scorers_dict_2d)
with open('top_scorers.pkl','wb') as tf:
    pickle.dump(top_scorers_dict_2d,tf)

{2008: {'SE Marsh': 616, 'G Gambhir': 534, 'ST Jayasuriya': 514, 'SR Watson': 472, 'GC Smith': 441}, 2009: {'ML Hayden': 572, 'AC Gilchrist': 495, 'AB de Villiers': 465, 'SK Raina': 434, 'TM Dilshan': 418}, 2010: {'SR Tendulkar': 618, 'JH Kallis': 572, 'SK Raina': 528, 'SC Ganguly': 493, 'M Vijay': 458}, 2011: {'CH Gayle': 608, 'V Kohli': 557, 'SR Tendulkar': 553, 'SE Marsh': 504, 'MEK Hussey': 492}, 2012: {'CH Gayle': 733, 'G Gambhir': 590, 'S Dhawan': 569, 'AM Rahane': 560, 'V Sehwag': 495}, 2013: {'MEK Hussey': 733, 'CH Gayle': 720, 'V Kohli': 639, 'SK Raina': 548, 'SR Watson': 543}, 2014: {'RV Uthappa': 660, 'DR Smith': 566, 'GJ Maxwell': 552, 'DA Warner': 528, 'SK Raina': 523}, 2015: {'DA Warner': 562, 'AM Rahane': 540, 'LMP Simmons': 540, 'AB de Villiers': 513, 'V Kohli': 505}, 2016: {'V Kohli': 973, 'DA Warner': 848, 'AB de Villiers': 687, 'G Gambhir': 501, 'S Dhawan': 501}, 2017: {'DA Warner': 641, 'G Gambhir': 498, 'S Dhawan': 479, 'SPD Smith': 472, 'SK Raina': 442}, 2018: {'K

In [95]:
ball2ball.columns

Index(['id', 'year', 'innings', 'overs', 'ball_number', 'batter', 'bowler',
       'non_striker', 'extra_type', 'batsman_run', 'extras_run', 'total_run',
       'non_boundary', 'iswicket_delivery', 'player_out', 'dismisal_kind',
       'fielders_involved', 'batting_team'],
      dtype='object')

In [ ]:
grouped_df = ball2ball.groupby(['year', 'bowler'])['iswicket_delivery'].count().reset_index()

grouped_df
top_scorers_df = grouped_df.groupby('year').apply(lambda x: x.nlargest(5, 'iswicket_delivery')).reset_index(drop=True)
# Print or use the resulting DataFrame 'grouped_df'
# top_scorers_dict = top_scorers_df.to_dict(orient='records')

# Print or use the resulting dictionary 'top_scorers_dict'
top_scorers_dict_2d = {}

# Iterate through the rows of the DataFrame and populate the 2D dictionary
for index, row in top_scorers_df.iterrows():
    year = row['year']
    batter = row['bowler']
    batsman_run = row['batsman_run']

    # If the year is not in the dictionary, add it
    if year not in top_scorers_dict_2d:
        top_scorers_dict_2d[year] = {}

    # Add the batter and batsman_run to the dictionary for the specific year
    top_scorers_dict_2d[year][batter] = batsman_run

# Print or use the resulting 2D dictionary 'top_scorers_dict_2d'
print(top_scorers_dict_2d)
with open('top_scorers.pkl','wb') as tf:
    pickle.dump(top_scorers_dict_2d,tf)


In [ ]:
ge = open('../data/player_stats/top_bowlers_by_season_corrected_economy.pkl','rb')
me = open('../data/player_stats/best_economies.pkl','rb')
ce = open('../data/player_stats/top_bowlers_by_season_int_year.pkl','rb')
gpt_economy = pickle.load(ge)
my_economy = pickle.load(me)
corrected_economy = pickle.load(ce)

In [ ]:
ball2ball['batter']

0         YBK Jaiswal
1         YBK Jaiswal
2          JC Buttler
3         YBK Jaiswal
4         YBK Jaiswal
             ...     
225949        P Kumar
225950       SB Joshi
225951        P Kumar
225952       SB Joshi
225953       SB Joshi
Name: batter, Length: 225954, dtype: object

In [ ]:
ball2ball.columns

Index(['id', 'year', 'innings', 'overs', 'ball_number', 'batter', 'bowler',
       'non_striker', 'extra_type', 'batsman_run', 'extras_run', 'total_run',
       'non_boundary', 'iswicket_delivery', 'player_out', 'dismisal_kind',
       'fielders_involved', 'batting_team'],
      dtype='object')

0         1
1         1
2         1
3         1
4         1
         ..
225949    2
225950    2
225951    2
225952    2
225953    2
Name: innings, Length: 225954, dtype: int64

In [ ]:
outs = ball2ball[ball2ball['iswicket_delivery'] == 1].groupby(['year','player_out'])['iswicket_delivery'].count().reset_index()

In [ ]:
runs = ball2ball.groupby(['year', 'batter']).agg(
    balls_faced =('batsman_run', 'size'),
    total_runs =('batsman_run', 'sum')
).reset_index()

,year,batter,balls_faced,total_runs
0,2008,A Chopra,55,42
1,2008,A Kumble,17,13
2,2008,A Mishra,42,37
3,2008,A Mukund,1,0
4,2008,A Nehra,13,3
...,...,...,...,...
2264,2022,WP Saha,266,317
2265,2022,Washington Sundar,76,101
2266,2022,YBK Jaiswal,198,258
2267,2022,YS Chahal,8,5


In [ ]:
result = pd.merge(runs, outs, left_on=['batter','year'], right_on=['player_out','year'], how='left')
result = result.dropna(subset='iswicket_delivery')

In [ ]:
result['batting_average'] = round(result['total_runs']/result['iswicket_delivery'],2)
result['strike_rate'] = round(result['total_runs']/result['balls_faced'],2)*100
result

In [ ]:
def convert_group_to_dict(group):
    return {row['batter']: {'runs': row['total_runs'], 'batting_average': row['batting_average'], 'strike_rate': row['strike_rate'], 'balls_faced': row['balls_faced']} for index, row in group.iterrows()}

# Group by 'year' and convert each group into the required dictionary format
result_dict = {year: convert_group_to_dict(group) for year, group in result.groupby('year')}

In [106]:
pickle_file = '../data/player_stats/batsman_metrics.pkl'
with open(pickle_file, 'wb') as file:
    pickle.dump(result_dict, file)

In [47]:
ball2ball['dismisal_kind'].unique()

array([nan, 'caught', 'caught and bowled', 'run out', 'bowled', 'stumped',
       'lbw', 'hit wicket', 'retired hurt', 'retired out',
       'obstructing the field'], dtype=object)

In [99]:
from collections import defaultdict
bowler_stats = {}
accepted = ['caught', 'caught and bowled', 'bowled', 'stumped','lbw', 'hit wicket']
for index, row in ball2ball.iterrows():
    bowler = row['bowler']
    year = row['year']
    extra_type = row['extra_type']
    total_run = row['total_run']
    extras_run = row['extras_run']
    iswicket = 1 if row['iswicket_delivery'] == 1 and row['dismisal_kind'] in accepted else 0
    if year not in bowler_stats:
        bowler_stats[year] = {}
    if bowler not in bowler_stats[year]:
        bowler_stats[year][bowler] = {}
        bowler_stats[year][bowler]['runs_conceeded'] = 0
        bowler_stats[year][bowler]['balls_bowled'] = 0
        bowler_stats[year][bowler]['wickets_taken'] = 0
    bowler_stats[year][bowler]['runs_conceeded'] += total_run - extras_run if extra_type in ['byes','legbyes'] else total_run
    bowler_stats[year][bowler]['balls_bowled'] += 1 if extra_type not in ['wides', 'noballs', 'penalty'] else 0
    bowler_stats[year][bowler]['wickets_taken'] += iswicket

In [100]:
for year in bowler_stats:
    for player in bowler_stats[year]:
        bowler_stats[year][player]['economy'] = bowler_stats[year][player]['runs_conceeded'] * 6 / bowler_stats[year][player]['balls_bowled']

In [102]:
file = '../data/player_stats/bowler_metrics.pkl'
with open(file,'wb') as bmf:
    pickle.dump(bowler_stats, bmf)

In [38]:
df = ball2ball.dropna(subset = 'fielders_involved')
df = df.groupby(['fielders_involved','year'])['innings'].count().reset_index()

,fielders_involved,year,innings
0,A Ashish Reddy,2012,1
1,A Ashish Reddy,2013,6
2,A Ashish Reddy,2015,2
3,A Ashish Reddy,2016,1
4,A Badoni,2022,9
...,...,...,...
1960,Z Khan,2010,4
1961,Z Khan,2011,4
1962,Z Khan,2012,5
1963,Z Khan,2016,3


In [119]:
"""
metrics to capture for teams every year
wins, loss, boundaries (4/6), runs, wickets
metrics to capture over all years
championships, playoffs
"""
with open('../data/team_stats/winloss.pkl', 'rb') as wl :
    d = pickle.load(wl)
    print(d)



{'2022': {'Rajasthan Royals': {'wins': 10, 'losses': 7}, 'Gujarat Titans': {'wins': 12, 'losses': 4}, 'Royal Challengers Bangalore': {'wins': 9, 'losses': 7}, 'Lucknow Super Giants': {'wins': 9, 'losses': 6}, 'Sunrisers Hyderabad': {'wins': 6, 'losses': 8}, 'Punjab Kings': {'wins': 7, 'losses': 7}, 'Delhi Capitals': {'wins': 7, 'losses': 7}, 'Mumbai Indians': {'wins': 4, 'losses': 10}, 'Chennai Super Kings': {'wins': 4, 'losses': 10}, 'Kolkata Knight Riders': {'wins': 6, 'losses': 8}}, '2021': {'Chennai Super Kings': {'wins': 11, 'losses': 5}, 'Kolkata Knight Riders': {'wins': 9, 'losses': 8}, 'Delhi Capitals': {'wins': 10, 'losses': 6}, 'Royal Challengers Bangalore': {'wins': 9, 'losses': 6}, 'Mumbai Indians': {'wins': 7, 'losses': 7}, 'Sunrisers Hyderabad': {'wins': 3, 'losses': 11}, 'Rajasthan Royals': {'wins': 5, 'losses': 9}, 'Punjab Kings': {'wins': 6, 'losses': 8}}, '2020/21': {'Delhi Capitals': {'wins': 9, 'losses': 8}, 'Mumbai Indians': {'wins': 11, 'losses': 5}, 'Sunrisers Hy

In [125]:
nd = {}
nk = 2008
for key in d:
    nd[nk] = d[key]
    nk += 1
with open('../data/team_stats/winloss.pkl', 'wb') as wl:
    pickle.dump(nd, wl)

In [167]:
x ={'Mumbai Indians': [0, 75, 160],
 'Chennai Super Kings': [255, 255, 60],
  'Kolkata Knight Riders': [46, 8, 84],
   'Sunrisers Hyderabad': [255, 130, 42],
    'Rajasthan Royals': [255, 192, 203],
     'Royal Challengers Bangalore': [236, 28, 36], 
     'Delhi Capitals': [0, 0, 139],
      'Punjab Kings': [220, 221, 223],
      'Gujarat Titans': [27, 33, 51],
       'Lucknow Super Giants': [129, 188, 100],
        'Kings XI Punjab': [220, 221, 223],
         'Delhi Daredevils': [0, 0, 139],
          'Rising Pune Supergiant': [158, 52, 149],
          'Rising Pune Supergiants': [158, 52, 149],
           'Pune Warriors': [0, 0, 0],
            'Deccan Chargers': [100, 136, 180],
             'Kochi Tuskers Kerala': [128, 0, 128],
              'Gujarat Lions': [255, 165, 0]}
with open('../data/team_stats/team_colors.pkl', 'wb') as wl :
    pickle.dump(x, wl)


In [171]:
team_colors = {}
for team in x:
    team_colors[team] = 'rgb('+str(tuple(x[team]))+')'
with open('../data/team_stats/team_colors.pkl', 'wb') as wl :
    pickle.dump(team_colors, wl)

In [172]:
team_colors

{'Mumbai Indians': 'rgb((0, 75, 160))',
 'Chennai Super Kings': 'rgb((255, 255, 60))',
 'Kolkata Knight Riders': 'rgb((46, 8, 84))',
 'Sunrisers Hyderabad': 'rgb((255, 130, 42))',
 'Rajasthan Royals': 'rgb((255, 192, 203))',
 'Royal Challengers Bangalore': 'rgb((236, 28, 36))',
 'Delhi Capitals': 'rgb((0, 0, 139))',
 'Punjab Kings': 'rgb((220, 221, 223))',
 'Gujarat Titans': 'rgb((27, 33, 51))',
 'Lucknow Super Giants': 'rgb((129, 188, 100))',
 'Kings XI Punjab': 'rgb((220, 221, 223))',
 'Delhi Daredevils': 'rgb((0, 0, 139))',
 'Rising Pune Supergiant': 'rgb((158, 52, 149))',
 'Rising Pune Supergiants': 'rgb((158, 52, 149))',
 'Pune Warriors': 'rgb((0, 0, 0))',
 'Deccan Chargers': 'rgb((100, 136, 180))',
 'Kochi Tuskers Kerala': 'rgb((128, 0, 128))',
 'Gujarat Lions': 'rgb((255, 165, 0))'}

KeyError: "None of [Index(['stadium'], dtype='object')] are in the [columns]"

In [168]:
with open('../data/team_stats/team_colors.pkl', 'rb') as tc :
    d = pickle.load(tc)
    print(d)

{'Mumbai Indians': [0, 75, 160], 'Chennai Super Kings': [255, 255, 60], 'Kolkata Knight Riders': [46, 8, 84], 'Sunrisers Hyderabad': [255, 130, 42], 'Rajasthan Royals': [255, 192, 203], 'Royal Challengers Bangalore': [236, 28, 36], 'Delhi Capitals': [0, 0, 139], 'Punjab Kings': [220, 221, 223], 'Gujarat Titans': [27, 33, 51], 'Lucknow Super Giants': [129, 188, 100], 'Kings XI Punjab': [220, 221, 223], 'Delhi Daredevils': [0, 0, 139], 'Rising Pune Supergiant': [158, 52, 149], 'Rising Pune Supergiants': [158, 52, 149], 'Pune Warriors': [0, 0, 0], 'Deccan Chargers': [100, 136, 180], 'Kochi Tuskers Kerala': [128, 0, 128], 'Gujarat Lions': [255, 165, 0]}


In [130]:
x = {'Mumbai Indians': {'championships': 5, 'playoff_appearances': 9},
 'Chennai Super Kings': {'championships': 4, 'playoff_appearances': 11},
  'Kolkata Knight Riders': {'championships': 2, 'playoff_appearances': 7},
   'Sunrisers Hyderabad': {'championships': 1, 'playoff_appearances': 6},
    'Rajasthan Royals': {'championships': 1, 'playoff_appearances': 5},
     'Deccan Chargers': {'championships': 1, 'playoff_appearances': 2},
      'Gujarat Titans': {'championships': 1, 'playoff_appearances': 1},
       'Royal Challengers Bangalore': {'championships': 0, 'playoff_appearances': 8},
        'Delhi Capitals': {'championships': 0, 'playoff_appearances': 3},
         'Delhi Daredevils': {'championships': 0, 'playoff_appearances': 3},
          'Kings XI Punjab': {'championships': 0, 'playoff_appearances': 2},
           'Lucknow Super Giants': {'championships': 0, 'playoff_appearances': 1},
            'Rising Pune Supergiant': {'championships': 0, 'playoff_appearances': 1},
             'Gujarat Lions': {'championships': 0, 'playoff_appearances': 1},
                'Pune Warriors': {'championships': 0, 'playoff_appearances': 0}, 
                'Kochi Tuskers Kerala': {'championships': 0, 'playoff_appearances': 0}}

with open('../data/team_stats/championships.pkl', 'wb') as ch :
    pickle.dump(x, ch)

In [134]:
matches['venue'].unique()

array(['Narendra Modi Stadium, Ahmedabad', 'Eden Gardens, Kolkata',
       'Wankhede Stadium, Mumbai', 'Brabourne Stadium, Mumbai',
       'Dr DY Patil Sports Academy, Mumbai',
       'Maharashtra Cricket Association Stadium, Pune',
       'Dubai International Cricket Stadium', 'Sharjah Cricket Stadium',
       'Zayed Cricket Stadium, Abu Dhabi', 'Arun Jaitley Stadium, Delhi',
       'MA Chidambaram Stadium, Chepauk, Chennai', 'Sheikh Zayed Stadium',
       'Rajiv Gandhi International Stadium',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'MA Chidambaram Stadium',
       'Punjab Cricket Association IS Bindra Stadium', 'Wankhede Stadium',
       'M.Chinnaswamy Stadium', 'Arun Jaitley Stadium', 'Eden Gardens',
       'Sawai Mansingh Stadium',
       'Maharashtra Cricket Association Stadium',
       'Holkar Cricket Stadium',
       'Rajiv Gandhi International Stadium, Uppal',
       'M Chinnaswamy Stadium', 'Feroz Shah Kotla', 'Green Park',
       'Punjab Cricket A

In [139]:
matches[['id','team1','team2']]

,id,team1,team2
0,1312200,Rajasthan Royals,Gujarat Titans
1,1312199,Royal Challengers Bangalore,Rajasthan Royals
2,1312198,Royal Challengers Bangalore,Lucknow Super Giants
3,1312197,Rajasthan Royals,Gujarat Titans
4,1304116,Sunrisers Hyderabad,Punjab Kings
...,...,...,...
945,335986,Kolkata Knight Riders,Deccan Chargers
946,335985,Mumbai Indians,Royal Challengers Bangalore
947,335984,Delhi Daredevils,Rajasthan Royals
948,335983,Kings XI Punjab,Chennai Super Kings


In [152]:
result = pd.merge(ball2ball, matches[['id','team1','team2']], on='id', how='inner')

In [160]:
team_stats = {}
for ind, row in result.iterrows():
    year = row['year']
    batting_team = row['batting_team']
    bowling_team = row['team2'] if row['batting_team'] == row['team1'] else row['team1']
    is_boundary = 1 if row['batsman_run'] >= 4 else 0 
    is_wicket = row['iswicket_delivery']
    runs = row['total_run']
    if year not in team_stats:
        team_stats[year] = {}
    if batting_team not in team_stats[year]:
        team_stats[year][batting_team]= {'boundaries': 0, 'wickets_taken': 0, 'runs_scored': 0}
        
    if bowling_team not in team_stats[year]:
       team_stats[year][bowling_team]= {'boundaries': 0, 'wickets_taken': 0, 'runs_scored': 0}
    
    team_stats[year][batting_team]['boundaries'] += is_boundary
    team_stats[year][batting_team]['runs_scored'] += runs
    team_stats[year][bowling_team]['wickets_taken'] += is_wicket
    
    
    

In [162]:
with open('../data/team_stats/team_stats.pkl', 'wb') as ts :
    pickle.dump(team_stats, ts)

In [159]:
ball2ball[ball2ball['non_boundary'] == 1]

,id,year,innings,overs,ball_number,batter,bowler,non_striker,extra_type,batsman_run,extras_run,total_run,non_boundary,iswicket_delivery,player_out,dismisal_kind,fielders_involved,batting_team
4632,1304101,2022,1,11,2,DP Conway,MR Marsh,S Dube,NaN,4,0,4,1,0,NaN,NaN,NaN,Chennai Super Kings
10604,1304076,2022,1,2,6,JC Buttler,UT Yadav,D Padikkal,NaN,4,0,4,1,0,NaN,NaN,NaN,Rajasthan Royals
13067,1304066,2022,1,0,3,D Padikkal,PVD Chameera,JC Buttler,NaN,4,0,4,1,0,NaN,NaN,NaN,Rajasthan Royals
30579,1254065,2021,2,11,3,F du Plessis,JA Richardson,MM Ali,NaN,4,0,4,1,0,NaN,NaN,NaN,Chennai Super Kings
32612,1237180,2020,1,6,4,S Dhawan,Rashid Khan,MP Stoinis,NaN,6,0,6,1,0,NaN,NaN,NaN,Delhi Capitals
45256,1216539,2020,1,15,1,RR Pant,SM Curran,SS Iyer,NaN,4,0,4,1,0,NaN,NaN,NaN,Delhi Capitals
48076,1178430,2019,2,13,2,N Pooran,DJ Bravo,Mandeep Singh,NaN,4,0,4,1,0,NaN,NaN,NaN,Kings XI Punjab
70980,1136580,2018,2,9,2,KS Williamson,SR Watson,Shakib Al Hasan,NaN,4,0,4,1,0,NaN,NaN,NaN,Sunrisers Hyderabad
86889,1082601,2017,1,18,6,MM Sharma,TA Boult,VR Aaron,NaN,4,0,4,1,0,NaN,NaN,NaN,Kings XI Punjab
88601,1082594,2017,2,12,4,GJ Maxwell,RD Chahar,DA Miller,NaN,4,0,4,1,0,NaN,NaN,NaN,Kings XI Punjab
